# This a code for batch georeferencing images 
* Rename files
* Batch georeference
* Merge(stick)

In [1]:
#Batch georeference images 
import os 
import rasterio
import cv2

In [4]:
src_images = []
src_dir = './patches' #add path to georeferenced patches
for filename in os.listdir(src_dir): 
    if os.path.splitext(filename)[1].lower() in ['.tif','.TIF', '.PNG', '.png', '.tiff']:
        src_images.append(os.path.join(src_dir, filename))
nongeoref_images = []
target_dir = './prediction' # add path to prediction patches
for filename1 in os.listdir(target_dir):  
    if os.path.splitext(filename1)[1].lower() in ['.tif', '.PNG', '.png', '.tiff']: 
        nongeoref_images.append(os.path.join(target_dir, filename1))

#outpath_georef = f'D:\\paper\\test\\site2_{set}_georef'
outpath_georef = './georeferenced' # add path to where georeferenced prediction patches are to be stored
# Ensure the output directory exists
os.makedirs(outpath_georef, exist_ok=True)
ext = ".tif"

for georef_image in src_images:
    georef_image_name = georef_image.split(os.sep)[-1]
    sID = georef_image_name.split(".")[0]
    #print(sID)
    georef = rasterio.open(georef_image)
    
    for predict_image in nongeoref_images:
        predict_image_name = predict_image.split(os.sep)[-1]
        dsID = predict_image_name.split(".")[0]
        #print (dsID)
        
        if dsID == sID:
            toref = cv2.imread(predict_image, 0)
            new_tif = rasterio.open(os.path.join(outpath_georef, sID + ext), 'w',
                                    driver='Gtiff',
                                    height=georef.height,
                                    width=georef.width,
                                    count=1,
                                    nodata=0,
                                    crs=georef.crs,
                                    transform=georef.transform,
                                    dtype='uint8')
            new_tif.write(toref, indexes=1)
            new_tif.close()

    georef.close()

print("Processing complete.")

Processing complete.


In [5]:
#Merge geotiff images 
import os
import rasterio
from rasterio.merge import merge
from rasterio import Affine
from rasterio.warp import calculate_default_transform, reproject
#os.chdir('')
# Specify the directory containing your GeoTIFF mask images
input_images = outpath_georef

# List all GeoTIFF files in the input folder
tif_files = [os.path.join(input_images, file) for file in os.listdir(input_images) if file.endswith('.tif')]

# Read and merge the GeoTIFF files
datasets = [rasterio.open(tif_file) for tif_file in tif_files]
merged, transform = merge(datasets)

# Define the output file path
#output_tif = f'C:/dl/ruhengeri/georef/merged_roof.tif'
output_tif = './georeferenced/merged_patches.tif'
# Update the metadata of the merged GeoTIFF
merged_meta = datasets[0].meta.copy()
merged_meta.update({
    'width': merged.shape[2],
    'height': merged.shape[1],
    'transform': transform
})

# Create the output GeoTIFF file and write the merged data
with rasterio.open(output_tif, 'w', bigtiff="yes", **merged_meta) as dest:
    dest.write(merged)

### (batch) Vectorise (convert raster to vector to create shapefile of buildings with predicted wealth classes)

In [9]:
import os
import rasterio
from rasterio import features
import geopandas as gpd
from shapely.geometry import shape

# Input and output directories
input_folder = './georeferenced' # Replace with your folder path
output_folder = "./georeferenced"  # Replace with your folder path

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all .tif files in the folder
for file in os.listdir(input_folder):
    if file.endswith(".tif"):  # Process only .tif files
        raster_path = os.path.join(input_folder, file)
        shapefile_name = os.path.splitext(file)[0] + ".shp"
        shapefile_path = os.path.join(output_folder, shapefile_name)

        print(f"Processing: {file}")

        # Open the raster file
        with rasterio.open(raster_path) as src:
            # Read the raster data (first band)
            image = src.read(1)
            mask = image != src.nodata  # Mask NoData values

            # Generate polygons
            results = features.shapes(image, mask=mask, transform=src.transform)

            # Create list of geometries and values
            polygons = []
            for geom, value in results:
                if value != src.nodata:
                    polygons.append({
                        'geometry': shape(geom),
                        'value': value
                    })

            # Skip if no valid polygons found
            if not polygons:
                print(f"Skipping {file} (No valid data)")
                continue

            # Create a GeoDataFrame with the raster's CRS
            gdf = gpd.GeoDataFrame(polygons, crs=src.crs)

            # Save to a shapefile
            gdf.to_file(shapefile_path)
            print(f"Saved: {shapefile_path}")

#print("Processing complete!")

Processing: prediction_Raster.tif
Saved: ./georeferenced/prediction_vectorised.shp
